In [ ]:
# Setup and Imports

import sys
curr_dir = '/user_data/csimmon2/git_repos/ptoc'
sys.path.insert(0, curr_dir)

import numpy as np
from nilearn import image
from scipy import stats
import pandas as pd
import os
from tqdm import tqdm
import warnings
import subprocess

# Define parameters
study_dir = "/lab_data/behrmannlab/vlad/ptoc"
raw_dir = "/lab_data/behrmannlab/vlad/hemispace"
results_dir = "/user_data/csimmon2/git_repos/ptoc/results"

# Load subject info
sub_info = pd.read_csv(f'{curr_dir}/sub_info.csv')
subs = sub_info[sub_info['group'] == 'control']['sub'].tolist()

# Define paths for input files
pIPS_path = f"{study_dir}/{{subject_id}}/ses-01/derivatives/fc/{{subject_id}}_pIPS_{{hemi}}_loc_fc.nii.gz"
LO_path = f"{study_dir}/{{subject_id}}/ses-01/derivatives/fc/{{subject_id}}_LO_{{hemi}}_loc_fc.nii.gz"

# Define output directories
group_out_dir = f'{curr_dir}/analyses/fc_subtraction'
os.makedirs(group_out_dir, exist_ok=True)

In [ ]:
# Subtraction

def load_and_subtract_fc_maps(pIPS_path, LO_path):
    pIPS_map = image.load_img(pIPS_path)
    LO_map = image.load_img(LO_path)
    difference_map = image.math_img("img1 - img2", img1=LO_map, img2=pIPS_map)
    return difference_map

def subject_level_analysis(subject_id, pIPS_path, LO_path, output_dir, hemi):
    pIPS_file = pIPS_path.format(subject_id=subject_id, hemi=hemi)
    LO_file = LO_path.format(subject_id=subject_id, hemi=hemi)
    
    if not os.path.exists(pIPS_file) or not os.path.exists(LO_file):
        print(f"Warning: Files not found for subject {subject_id}, hemisphere {hemi}. Skipping.")
        return None

    difference_map = load_and_subtract_fc_maps(pIPS_file, LO_file)
    
    subtraction_file = os.path.join(output_dir, f'{subject_id}_LO_minus_pIPS_{hemi}_fc.nii.gz')
    difference_map.to_filename(subtraction_file)
    
    print(f"Subtraction complete for subject {subject_id}, hemisphere {hemi}. Result saved in {subtraction_file}")
    return subtraction_file

# Perform subject-level analysis and collect subtraction files
subtraction_files = {'left': [], 'right': []}

for sub in tqdm(subs, desc="Processing subjects"):
    sub_dir = f"{study_dir}/{sub}/ses-01"
    out_dir = f"{sub_dir}/derivatives"
    fc_dir = f"{out_dir}/fc"
    os.makedirs(fc_dir, exist_ok=True)

    for hemi in ['left', 'right']:
        subtraction_file = subject_level_analysis(sub, pIPS_path, LO_path, fc_dir, hemi)
        if subtraction_file:
            subtraction_files[hemi].append(subtraction_file)

print("Subtraction completed for all subjects.")

In [ ]:
# Conversion to Standard Space
# alt run sub2stand.py in terminal
#(fmri) (base) [csimmon2@mind-1-21-2 ptoc]$ module load fsl-6.0.3 
#(fmri) (base) [csimmon2@mind-1-21-2 ptoc]$ python analyses/fc_subtraction/sub2stand.py 

mni_brain = os.path.join(os.environ['FSLDIR'], "data/standard/MNI152_T1_2mm_brain.nii.gz")

def convert_to_mni(sub, input_file, output_file, anat2mni_mat, mni_brain):
    if not os.path.isfile(input_file):
        print(f"Input file not found: {input_file}")
        return None

    if os.path.isfile(output_file):
        print(f"MNI file already exists: {output_file}")
        return output_file

    print(f"Converting to MNI space for {sub}")
    subprocess.run([
        'flirt',
        '-in', input_file,
        '-ref', mni_brain,
        '-out', output_file,
        '-applyxfm',
        '-init', anat2mni_mat,
        '-interp', 'trilinear'
    ], check=True)

    return output_file

mni_files = {'left': [], 'right': []}

for sub in tqdm(subs, desc="Converting to MNI space"):
    sub_dir = f"{study_dir}/{sub}/ses-01"
    out_dir = f"{sub_dir}/derivatives"
    fc_dir = f"{out_dir}/fc"
    fc_mni_dir = f"{out_dir}/fc_mni"
    os.makedirs(fc_mni_dir, exist_ok=True)

    # Generate transformation matrix if it doesn't exist
    anat2mni_mat = f"{out_dir}/anat2mni.mat"
    if not os.path.isfile(anat2mni_mat):
        anat_brain = f"{raw_dir}/{sub}/ses-01/anat/{sub}_ses-01_T1w_brain.nii.gz"
        if not os.path.isfile(anat_brain):
            print(f"Anatomical image not found for {sub}. Skipping...")
            continue
        print(f"Generating transformation matrix for {sub}")
        subprocess.run([
            'flirt',
            '-in', anat_brain,
            '-ref', mni_brain,
            '-omat', anat2mni_mat,
            '-bins', '256',
            '-cost', 'corratio',
            '-searchrx', '-90', '90',
            '-searchry', '-90', '90',
            '-searchrz', '-90', '90',
            '-dof', '12'
        ], check=True)

    for hemi in ['left', 'right']:
        input_file = f"{fc_dir}/{sub}_LO_minus_pIPS_{hemi}_fc.nii.gz"
        mni_file = f"{fc_mni_dir}/{sub}_LO_minus_pIPS_{hemi}_fc_mni.nii.gz"
        mni_result = convert_to_mni(sub, input_file, mni_file, anat2mni_mat, mni_brain)
        if mni_result:
            mni_files[hemi].append(mni_result)

print("Conversion to MNI space completed for all subjects.")

In [ ]:
# Statistics

# Cell 4: Statistics

import os
import numpy as np
from nilearn import image
import pandas as pd
from tqdm import tqdm

# Setup
curr_dir = '/user_data/csimmon2/git_repos/ptoc'
study_dir = "/lab_data/behrmannlab/vlad/ptoc"

# Load subject info
sub_info = pd.read_csv(f'{curr_dir}/sub_info.csv')
subs = sub_info[sub_info['group'] == 'control']['sub'].tolist()

# Define output directory for group analysis
group_out_dir = f'{curr_dir}/analyses/fc_subtraction'
os.makedirs(group_out_dir, exist_ok=True)

def safe_divide(a, b):
    with np.errstate(divide='ignore', invalid='ignore'):
        c = np.true_divide(a, b)
        c[~np.isfinite(c)] = 0  # replace inf/NaN with 0
    return c

def group_level_analysis(mni_files, output_dir, hemi):
    if not mni_files:
        print(f"Error: No valid MNI space files for {hemi} hemisphere. Skipping t-test.")
        return

    print(f"Processing {len(mni_files)} files for {hemi} hemisphere")

    # Load all MNI space maps
    all_maps = [image.load_img(f) for f in mni_files]

    # Stack all maps into a 4D image
    stacked_maps = image.concat_imgs(all_maps)

    # Perform voxel-wise one-sample t-test
    print(f"Performing t-test for {hemi} hemisphere")
    data = stacked_maps.get_fdata()
    mean = np.mean(data, axis=-1)
    std = np.std(data, axis=-1)
    n = data.shape[-1]
    t_values = safe_divide(mean, (std / np.sqrt(n)))

    # Create and save the t-map
    t_map = image.new_img_like(all_maps[0], t_values)
    t_map_file = os.path.join(output_dir, f'group_ttest_LO_minus_pIPS_{hemi}.nii.gz')
    t_map.to_filename(t_map_file)

    print(f"Group-level analysis complete for {hemi} hemisphere. T-map saved in {t_map_file}")

# Collect MNI files
mni_files = {'left': [], 'right': []}

for sub in tqdm(subs, desc="Collecting MNI files"):
    sub_dir = f"{study_dir}/{sub}/ses-01"
    fc_mni_dir = f"{sub_dir}/derivatives/fc_mni"
    
    for hemi in ['left', 'right']:
        mni_file = f"{fc_mni_dir}/{sub}_LO_minus_pIPS_{hemi}_fc_mni.nii.gz"
        if os.path.exists(mni_file):
            mni_files[hemi].append(mni_file)
        else:
            print(f"Warning: MNI file not found for subject {sub}, hemisphere {hemi}")

# Perform group-level analysis
for hemi in ['left', 'right']:
    group_level_analysis(mni_files[hemi], group_out_dir, hemi)

print("Analysis completed for both hemispheres.")